In [ ]:
import numpy as np
import sympy as sp
from scipy.optimize import minimize
from scipy.spatial import cKDTree
import plotly.graph_objects as go
from lower_hull import lower_hull
from symbolic_hulls import *

### Can we compute the boundary equations on higher order energy functions?

In [ ]:
# Define variables
x1, x2= sp.symbols('x1 x2')

E1 = x1**4 + 2*x2**4 + 5*x1**2*x2**2 - 6*x2**2 + 9*x1*x2

# Define second polynomial
E2 = 2*x1**4 - 3*x2**4 + 6*x1**2 + 4*x1**2*x2**2 - x1*x2

# Display the polynomials
display(E1, E2)

In [ ]:
proj, variables, pvariables = projection_function(E1, E2)
disc1 = recursive_discriminant(proj, variables)

proj, variables, pvariables = projection_function(E2, E1)
disc2 = recursive_discriminant(proj, variables)

In [ ]:
### THIS TOOK OVER AN HOUR TO RUN AND DIDNT FINISH
# x3 = sp.symbols('x3')
# # Define first polynomial
# E1 = x1**4 + 2*x2**4 - 3*x3**4 + 5*x1**2*x2**2 - 6*x2**2*x3**2 + 7*x3**2 + 9*x1*x2*x3

# # Define second polynomial
# E2 = 2*x1**4 - 3*x2**4 + x3**4 + 6*x1**2*x3**2 + 4*x1**2*x2**2 - x1*x2*x3

# proj, variables, pvariables = projection_function(E1, E2)
# disc1 = recursive_discriminant(proj, variables)

# proj, variables, pvariables = projection_function(E2, E1)
# disc2 = recursive_discriminant(proj, variables)

# display(disc1, disc2)

In [ ]:
# Create a grid of points in [-5,5] x [-5,5], for example
grid_size = 50
x_vals = np.linspace(-5, 5, grid_size)
y_vals = np.linspace(-5, 5, grid_size)
X, Y = np.meshgrid(x_vals, y_vals)

# Evaluate disc1_func and disc2_func on the grid
Z1 = np.zeros_like(X, dtype=float)
Z2 = np.zeros_like(X, dtype=float)

E1_func = sp.lambdify((x1, x2), E1, 'numpy')
E2_func = sp.lambdify((x1, x2), E2, 'numpy')
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z1[i, j] = E1_func(X[i, j], Y[i, j])
        Z2[i, j] = E2_func(X[i, j], Y[i, j])

fig_discs = go.Figure()

# Surface for disc1
fig_discs.add_trace(go.Surface(
    x=X,
    y=Y,
    z=Z1,
    name='disc1'
))

# Surface for disc2
fig_discs.add_trace(go.Surface(
    x=X,
    y=Y,
    z=Z2,
    name='disc2'
))

fig_discs.update_layout(
    title="3D Surfaces for disc1 and disc2",
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='disc values'
    )
)

fig_discs.show()

### Can we compute values on these boundaries?

In [ ]:
x1_p, x2_p = sp.symbols('x1_p x2_p')

disc1_func = sp.lambdify((x1_p, x2_p), disc1, 'numpy')
disc2_func = sp.lambdify((x1_p, x2_p), disc2, 'numpy')

In [ ]:
# Wrap the functions so that they can be called with a single argument
def wrapped_disc1(X):
    return disc1_func(*X)

def wrapped_disc2(X):
    return disc2_func(*X)

# Define an objective function that will be minimized to step the discriminants towards zeros
def objective1(x):
    # 1/2 * [f(x)]^2
    return 0.5 * wrapped_disc1(x)**2

def objective2(x):
    # 1/2 * [f(x)]^2
    return 0.5 * wrapped_disc2(x)**2

# Define the gradient of the objective function
psudo_disc1_grad = sp.lambdify((x1, x2), sp.Matrix([disc1]).jacobian([x1, x2]), "numpy")
psudo_disc2_grad = sp.lambdify((x1, x2), sp.Matrix([disc2]).jacobian([x1, x2]), "numpy")

def objective_grad1(X):
    f_val = wrapped_disc1(X)
    grad_f = psudo_disc1_grad(*X) 
    grad_f = np.array(grad_f).ravel()
    return f_val * grad_f

def objective_grad2(X):
    f_val = wrapped_disc2(X)
    grad_f = psudo_disc2_grad(*X)
    grad_f = np.array(grad_f).ravel()
    return f_val * grad_f

In [ ]:
# Define a space of guess points
space = np.random.uniform(low=-4.0, high=4.0, size=(100000, 2))

sols1 = []
for point in space:
    # Call the optimizer with max iterations
    res = minimize(
        objective1,
        point,
        jac=objective_grad1,
        method="BFGS",
    )
    if res.fun < 100:
        sols1.append(res.x)

sols1 = np.array(sols1)

sols2 = []
for point in space:
    # Call the optimizer with max iterations
    res = minimize(
        objective2,
        point,
        jac=objective_grad2,
        method="BFGS",
    )
    if res.fun < 100:
        sols2.append(res.x)

sols2 = np.array(sols2)

In [ ]:
print("sols1 shape:" , sols1.shape)
print("sols2 shape:" , sols2.shape)

### Lets use the trivial approach

In [ ]:
# Assume x1, x2, E1, E2 are defined somewhere
x1, x2 = sp.symbols('x1 x2')
E1 = x1**2 + x2**2
E2 = x1 - x2**3

# Compile once
E1_func = sp.lambdify((x1, x2), E1, 'numpy')
E2_func = sp.lambdify((x1, x2), E2, 'numpy')

# Generate space
space = np.random.uniform(low=-4.0, high=4.0, size=(10000, 2))

# Evaluate efficiently using unpacking
E1_vals = E1_func(space[:, 0], space[:, 1])
E2_vals = E2_func(space[:, 0], space[:, 1])

# Combine results
E1_points = np.column_stack((space, E1_vals))
E2_points = np.column_stack((space, E2_vals))

points = np.vstack((E1_points, E2_points))

In [ ]:
# Now run the lower convex hull algorithm
hull = lower_hull(points)